In [242]:
# Instalace a aktualizace modulů

# !pip install --upgrade pip
# !pip install --upgrade tabulate
# !pip install --upgrade forallpeople
# !pip install --upgrade handcalcs
# !pip install --upgrade pytexit
# !pip install --upgrade pyXSteam
# !pip install --upgrade CoolProp

In [243]:
# Import knihoven a modulů

import pandas as pd
from CoolProp.CoolProp import PropsSI
import handcalcs.render
from math import pi, e, sqrt, log
from pyXSteam.XSteam import XSteam
from tabulate import tabulate
import pyromat as pm

In [244]:
# Nastavení prostředí a modulů

%decimal_separator ,
steam = XSteam(XSteam.UNIT_SYSTEM_MKS) # Jednotky: m/kg/sec/°C/bar/W
pd.set_option("display.precision", 3)

pm.config['unit_temperature'] = 'C'

***
## Základní parametry kotle
***

In [245]:
%%render params 3
m_p = (76*1000)/3600        #kg/s
p_out = 3.1                 #MPa
t_out = 370                 #°C
t_nv = 71                   #°C
m_vst = 0.36                #kg/s
t_vst = 217                 #°C

<IPython.core.display.Latex object>

***
## Parametry spalin ze spalovací turbíny
***

In [246]:
R = 8.314462618 #J/kg/mol
T = 273.15 #K

#### Objemové složení spalin, hmotnostní průtok a teplota na vstupu do hořáku

In [247]:
%%render params 3
m_sp = 130.1    #kg/s
t_sp = 562.0    #°C

# Objemové složení spalin
x_N2 = 0.7457
x_O2 = 0.1289
x_H2O = 0.0801
x_CO2 = 0.0364
x_Ar = 0.0089

<IPython.core.display.Latex object>

#### Molární hmotnosti spalin

In [248]:
%%render params 3

# Molární hmotnost složek spalin (z podkladů)
M_N2 = 28.0134e-03      #kg/mol
M_O2 = 31.9988e-03      #kg/mol
M_H2O = 18.0152e-03     #kg/mol
M_CO2 = 44.0095e-03     #kg/mol
M_Ar = 39.9480e-03      #kg/mol

<IPython.core.display.Latex object>

#### Hustota složek spalin (z podkladů)

In [249]:
%%render params
rho_N2 = 1.2504     #kg/m3
rho_O2 = 1.4290     #kg/m3
rho_H2O = 0.8038    #kg/m3
rho_CO2 = 1.9772    #kg/m3
rho_Ar = 1.7840     #kg/m3
rho_sp = x_N2*rho_N2 + x_O2*rho_O2 + x_H2O*rho_H2O + x_CO2*rho_CO2 + x_Ar*rho_Ar #kg/m3

<IPython.core.display.Latex object>

#### Hmotnostní procenta (-)

In [250]:
# Hmotnosti
m_N2 = x_N2*rho_N2                      #kg
m_O2 = x_O2*rho_O2                      #kg
m_H2O = x_H2O*rho_H2O                   #kg
m_CO2 = x_CO2*rho_CO2                   #kg
m_Ar = x_Ar*rho_Ar                      #kg

In [251]:
# Hmotnostní podíl
w_N2 = m_N2/rho_sp                        #kg
w_O2 = m_O2/rho_sp                        #kg
w_H2O = m_H2O/rho_sp                      #kg
w_CO2 = m_CO2/rho_sp                      #kg
w_Ar = m_Ar/rho_sp                        #kg

#### Objemový průtok spalin

In [252]:
%%render
V_sp = m_sp/rho_sp #m3n/s

<IPython.core.display.Latex object>

In [253]:
%%render params 3

# Měrný objem složek spalin (z podkladů)
v_N2 = 22.403     #m3/kg
v_O2 = 22.392     #m3/kg
v_H2O = 22.414    #m3/kg
v_CO2 = 22.258    #m3/kg
v_Ar = 22.393     #m3/kg

<IPython.core.display.Latex object>

#### Entalpie spalin (z modulu PyroMat - shodné s poskytnutými materiály)

In [254]:
N2 = pm.get('ig.N2')
O2 = pm.get('ig.O2')
H2O = pm.get('ig.H2O')
CO2 = pm.get('ig.CO2')
Ar = pm.get('ig.Ar')

In [255]:
def entalpie_sp(teplota, jednotka=''):

    """
    teplota = zadávat ve °C
    jednotka = volba mezi 'kg' / 'm3n'

    Vypočítá entalpii [kJ/kg nebo kJ/m3n] spalin při dané teplotě [°C] vzhledem k entalpii daného prvku při 0°C. Výstupem je dictionary. Pro zobrazení entalpie prvku  dané teploty je nutné vybrat daný prvek nebo spaliny ['N2', 'O2', 'H2O', 'CO2', 'Ar', 'sp']. Výstupem pro jeden konkrétní prvek je float.
    """

    h_N2 = float(N2.h(teplota)-N2.h(0)) #kJ/kg
    I_N2 = h_N2*rho_sp

    h_O2 = float(O2.h(teplota)-O2.h(0)) #kJ/kg
    I_O2 = h_O2*rho_sp

    h_H2O = float(H2O.h(teplota)-H2O.h(0)) #kJ/kg
    I_H2O = h_H2O*rho_sp

    h_CO2 = float(CO2.h(teplota)-CO2.h(0)) #kJ/kg
    I_CO2 = h_CO2*rho_sp

    h_Ar = float(Ar.h(teplota)-Ar.h(0)) #kJ/kg
    I_Ar = h_Ar*rho_sp

    h_spalin = w_N2*h_N2 + w_O2*h_O2 + w_H2O*h_H2O + w_CO2*h_CO2 + w_Ar*h_Ar #kJ/kg
    I_spalin = w_N2*I_N2 + w_O2*I_O2 + w_H2O*I_H2O + w_CO2*I_CO2 + w_Ar*I_Ar #kJ/m3n


    if jednotka == 'kg':
        return {'N2': h_N2, 'O2': h_O2, 'H2O': h_H2O , 'CO2': h_CO2, 'Ar': h_Ar, 'sp': h_spalin}
    elif jednotka == 'm3n':
        return {'N2': I_N2, 'O2': I_O2, 'H2O': I_H2O , 'CO2': I_CO2, 'Ar': I_Ar, 'sp': I_spalin}
    else:
        print('Nutno zadat jednotku - kg / m3n!')

Interpolace

In [256]:
def lambda_interpolate(teplota):
    t_min = (int(teplota/100))*100
    t_max = (int(teplota/100)+1)*100

    lambda_init = float(lambda_tabulka_print.loc[t_min,"8 % H2O"])
    delta_T = teplota-t_min
    delta_lambda = lambda_tabulka_print.loc[t_max,"8 % H2O"]-lambda_tabulka_print.loc[t_min,"8 % H2O"]
    vysledek = (lambda_init + delta_T*(delta_lambda/(t_max-t_min)))*10**-3
    return vysledek

In [257]:
def nu_interpolate(teplota):
    t_min = (int(teplota/100))*100
    t_max = (int(teplota/100)+1)*100

    nu_init = float(nu_tabulka_print.loc[t_min,"8 % H2O"])
    delta_T = teplota-t_min
    delta_nu = nu_tabulka_print.loc[t_max,"8 % H2O"]-nu_tabulka_print.loc[t_min,"8 % H2O"]
    vysledek = (nu_init + delta_T*(delta_nu/(t_max-t_min)))*10**-6
    return vysledek


In [258]:
def Pr_interpolate(teplota):
    t_min = (int(teplota/100))*100
    t_max = (int(teplota/100)+1)*100

    Pr_init = float(Pr_tabulka_print.loc[t_min,"8 % H2O"])
    delta_T = teplota-t_min
    delta_Pr = Pr_tabulka_print.loc[t_max,"8 % H2O"]-Pr_tabulka_print.loc[t_min,"8 % H2O"]
    vysledek = (Pr_init + delta_T*(delta_Pr/(t_max-t_min)))
    return vysledek

### <u>Bod A (Vstup spalin - za hořákem)

In [259]:
%%render
t_A = t_sp #°C
I_A_sp = entalpie_sp(t_A, 'm3n')['sp']  #kJ/m3n

<IPython.core.display.Latex object>

### Bod B (za druhým přehřívákem)

In [260]:
%%render
t_B = 549 #°C
I_B_sp = entalpie_sp(t_B,'m3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod C (za prvním přehřívákem)

In [261]:
%%render
t_C = 507 #°C
I_C_sp = entalpie_sp(t_C, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod D (za druhým výparníkem)

In [262]:
%%render
t_D = 366 #°C
I_D_sp = entalpie_sp(t_D, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod E (za prvním výparníkem)

In [263]:
%%render
t_E = 254 #°C
I_E_sp = entalpie_sp(t_E, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod F (za druhým ekonomizérem)

In [264]:
%%render
t_F = 241 #°C
I_F_sp = entalpie_sp(t_F, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

### Bod G (za prvním ekonomizérem, výstup spalin)

In [265]:
%%render
t_G = 145 #°C
I_G_sp = entalpie_sp(t_G, 'm3n')['sp'] #kJ/m3n

<IPython.core.display.Latex object>

***
## Výpočty ze strany vody a vodní páry
***

#### Bod 1 (Konec druhého přehříváku, výstupní pára)

In [266]:
%%render params
t_1 = t_out                     #°C
p_1 = p_out                     #MPa
h_1 = steam.h_pt(p_1*10,t_1)    #kJ/kg
v_1 = steam.v_pt(p_1*10,t_1)    #m3/kg
x_1 = steam.x_ph(p_1*10,h_1)    #-

<IPython.core.display.Latex object>

#### Bod 3 (Konec prvního přehříváku, před vstřikem)

In [267]:
%%render params
t_3 = 348                       #°C
p_3 = 3.33                      #MPa
h_3 = steam.h_pt(p_3*10,t_3)    #kJ/kg
v_3 = steam.v_pt(p_3*10,t_3)    #m3/kg
x_3 = steam.x_ph(p_3*10,h_3)    #-

<IPython.core.display.Latex object>

#### Bod 4 (Začátek prvního přehříváku, po druhém výparníku)

In [268]:
%%render params
p_4 = 3.38                      #MPa
t_4 = steam.tsat_p(p_4*10)      #°C
h_4 = steam.hV_p(p_4*10)        #kJ/kg
v_4 = steam.vV_p(p_4*10)        #m3/kg
x_4 = steam.x_ph(p_4*10,h_4)    #-

<IPython.core.display.Latex object>

#### Bod 5 (začátek prvního výparníku, kde je sytá kapalina)

In [269]:
%%render params
p_5 = 3.38                      #MPa
t_5 = steam.tsat_p(p_5*10)      #°C
h_5 = steam.hL_p(p_5*10)        #kJ/kg
v_5 = steam.vL_p(p_5*10)        #m3/kg
x_5 = steam.x_ph(p_5*10,h_5)    #-

<IPython.core.display.Latex object>

#### Bod 6 (Za posledním ekonomizérem, před vstupem do výparníku)

In [270]:
%%render params
p_6 = 3.68                      #MPa
t_6 = 236                       #°C
h_6 = steam.h_pt(p_6*10, t_6)   #kJ/kg
v_6 = steam.v_pt(p_6*10, t_6)   #m3/kg
x_6 = steam.x_ph(p_6*10,h_6)    #-

<IPython.core.display.Latex object>

#### Bod 7 (mezi ekonomizéry)

In [271]:
%%render params
p_7 = 3.74                      #MPa
t_7 = 217                       #°C
h_7 = steam.h_pt(p_7*10, t_7)   #kJ/kg
v_7 = steam.v_pt(p_7*10, t_7)   #m3/kg
x_7 = steam.x_ph(p_7*10,h_7)    #-


<IPython.core.display.Latex object>

#### Bod 2 (Začátek druhého přehříváku, po vstřiku)

In [272]:
%%render params
p_2 = p_3                           #MPa
h_2 = (20.74*h_3 + 0.36*h_7)/21.10  #kJ/kg
t_2 = steam.t_ph(p_2*10, h_2)       #°C
v_2 = steam.v_ph(p_2*10, h_2)       #m3/kg
x_2 = steam.x_ph(p_2*10,h_2)        #-

<IPython.core.display.Latex object>

#### Bod 8 (Před začátkem ekonomizéru, vstup napájecí vody)

In [273]:
%%render params
t_8 = 71                         #°C
p_8 = 4.2                        #MPa
h_8 = steam.h_pt(p_8*10,t_8)     #kJ/kg
v_8 = steam.v_pt(p_8*10,t_8)     #m3/kg
x_8 = steam.x_ph(p_8*10,h_8)     #-

<IPython.core.display.Latex object>

***
## Výkon kotle a jednotlivých teplosměnných ploch
***

### Maximální využitelný výkon

In [274]:
%%render
# Dodaný výkon v palivu
Q_in = V_sp*(I_A_sp-entalpie_sp(25, 'm3n')['sp']) #kW

<IPython.core.display.Latex object>

In [275]:
%%render
# Odebraný výkon vodou a párou
Q_out = m_p*h_1-m_p*h_8 #kW

<IPython.core.display.Latex object>

### Účinnost

In [276]:
%%render
eta = Q_out/Q_in #-

<IPython.core.display.Latex object>

### Předaná tepla v jednotlivých teplosměnných plochách

In [277]:
%%render
# Ztráta sáláním - pro výpočet Q_16
C = 0.0113 #-
Q_ZS = ((Q_in/1000)**0.7)*C*1000 #kW

<IPython.core.display.Latex object>

In [278]:
%%render
z_s = Q_ZS/Q_in #-

<IPython.core.display.Latex object>

#### Předané teplo v bodech A-E

In [279]:
%%render
Q_AE = V_sp*(I_A_sp-I_E_sp) #kW

<IPython.core.display.Latex object>

#### Předané teplo v bodech 1-6

In [280]:
%%render
Q_16 = Q_AE*(1-z_s)

<IPython.core.display.Latex object>

### Predane tepelne toky v jednotlivych teplosmennych plochach

#### Prehrivak 2

In [281]:
%%render
Q_P2 = m_p*(h_1-h_2) #kW

<IPython.core.display.Latex object>

#### Prehrivak 1

In [282]:
%%render
Q_P1 = (m_p-m_vst)*(h_3-h_4) #kW

<IPython.core.display.Latex object>

#### Vyparnik 1 + 2

In [283]:
%%render
Q_V = (m_p-m_vst)*(h_4-h_5) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 2

In [284]:
%%render
Q_EKO_2 = (m_p-m_vst)*(h_6-h_7) #kW

<IPython.core.display.Latex object>

#### Ekonomizer 1

In [285]:
%%render
Q_EKO_1 = m_p*(h_7-h_8) #kW

<IPython.core.display.Latex object>

***
# Návrh teplosměnných ploch
***

**Označení veličin**

D = vnější průměr

t<sub>tr</sub> = tloušťka trubky

d = vnitřní průměr

h<sub>ž</sub> = výška žebra

t<sub>ž</sub> = tloušťka žebra

s<sub>ž</sub> = rozteč žeber

n<sub>ž</sub> = počet žeber na 1 m<sup>2</sup>

s<sub>1</sub> = příčná rozteč

s<sub>2</sub> = podélná rozteč

## Přehřívák 2 (bez žeber)
***

In [286]:
%%render params
D = 0.0445      #m
t_tr = 0.0029   #m
d = D-2*t_tr    #m
h_z = 0.000     #m
t_z = 0.000     #m
s_z = 0.000     #m
n_z = 0         #1/m
s_2 = 0.117     #m

<IPython.core.display.Latex object>

In [287]:
prehrivak2_input = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z']
prehrivak2_table = pd.DataFrame(prehrivak2_input, index = zkratky)

In [288]:
prehrivak2_table

# print(prehrivak2_table.to_latex())

,Hodnota,Jednotka
D,44.5,mm
t_tr,2.9,mm
d,38.7,mm
h_z,0.0,mm
t_z,0.0,mm
s_z,0.0,mm
n_z,0.0,-


### Rozměry trubek

In [289]:
%%render
# zvoleno tak, aby rozteč byla stejná jako u přehříváku 1
a = 0.0435  #m

<IPython.core.display.Latex object>

In [290]:
%%render
s_1 = D+a   #m

<IPython.core.display.Latex object>

Počet trubek

In [291]:
sirka = 3   #m
H = 13.6    #m

In [292]:
%%render
n_tr = sirka/s_1-1/2

<IPython.core.display.Latex object>

In [293]:
n_tr = round(n_tr,0)

#### Pára

Střední hodnoty

In [294]:
%%render
t_21 = (t_1+t_2)/2  #°C
p_21 = (p_1+p_2)/2  #MPa

<IPython.core.display.Latex object>

In [295]:
%%render
v_21 = steam.v_pt(p_21*10,t_21)    #m3/kg

<IPython.core.display.Latex object>

Rychlost páry

In [296]:
%%render
w_p = 4*(m_p*v_21)/(pi*d**2*2*n_tr)   #m/s

<IPython.core.display.Latex object>

#### Spaliny

In [297]:
%%render
t_AB = (t_A+t_B)/2  #°C

<IPython.core.display.Latex object>

Skutečný objemový průtok spalin

In [298]:
%%render
V_sp_skut = V_sp * ((t_AB+273.15)/273.15)   #m3/s

<IPython.core.display.Latex object>

Skutečný průřez spalinového kanálu

In [299]:
%%render
S_kan = H*(sirka-n_tr*D)    #m2

<IPython.core.display.Latex object>

Rychlost spalin

In [300]:
%%render
w_sp = V_sp_skut/S_kan  #m/s

<IPython.core.display.Latex object>

### Součinitel přestupu tepla ze strany spalin

In [301]:
%%render
c_t = 1
c_l = 1
c_m = 1

<IPython.core.display.Latex object>

### Prostup tepla

#### Tabulka $\lambda$ pro obsah vody

In [302]:
lambda_0 = 22.5+(8-5)*((22.7-22.5)/(10-5))
lambda_100 = 30.5+(8-5)*((31.2-30.5)/(10-5))
lambda_200 = 38.7+(8-5)*((39.8-38.7)/(10-5))
lambda_300 = 46.4+(8-5)*((48.1-46.4)/(10-5))
lambda_400 = 54.3+(8-5)*((56.6-54.3)/(10-5))
lambda_500 = 62.2+(8-5)*((65.1-62.2)/(10-5))
lambda_600 = 70+(8-5)*((73.5-70)/(10-5))

lambda_tabulka = pd.DataFrame(
{
        "T [°C]": (0,100,200,300,400,500,600),
        "5 % H2O": (22.5,30.5,38.7,46.4,54.3,62.2,70.0),
        "8 % H2O": (lambda_0,lambda_100,lambda_200,lambda_300,lambda_400,lambda_500,lambda_600),
        "10 % H2O": (22.7,31.2,39.8,48.1,56.6,65.1,73.5),
    }
)
lambda_tabulka_print = lambda_tabulka.set_index(lambda_tabulka.columns[0])

In [303]:
lambda_tabulka_print

# print(lambda_tabulka_print.to_latex())

,5 % H2O,8 % H2O,10 % H2O
T [°C],,,
0,22.5,22.62,22.7
100,30.5,30.92,31.2
200,38.7,39.36,39.8
300,46.4,47.42,48.1
400,54.3,55.68,56.6
500,62.2,63.94,65.1
600,70.0,72.10,73.5


#### Tabulka $\nu$ pro daný obsah vody

In [304]:
nu_0 = 12.2+(8-5)*((12.2-12.2)/(10-5))
nu_100 = 21.3+(8-5)*((21.5-21.3)/(10-5))
nu_200 = 31.8+(8-5)*((32.8-31.8)/(10-5))
nu_300 = 45.0+(8-5)*((45.8-45.0)/(10-5))
nu_400 = 59.2+(8-5)*((60.4-59.2)/(10-5))
nu_500 = 74.6+(8-5)*((76.3-74.6)/(10-5))
nu_600 = 91.4+(8-5)*((93.6-91.4)/(10-5))

nu_tabulka = pd.DataFrame(
{
        "T [°C]": (0,100,200,300,400,500,600),
        "5 % H2O": (12.2,21.3,31.8,45.0,59.2,74.6,91.4),
        "8 % H2O": (nu_0,nu_100,nu_200,nu_300,nu_400,nu_500,nu_600),
        "10 % H2O": (12.2,21.5,32.8,45.8,60.4,76.3,93.6),
    }
)
nu_tabulka_print = nu_tabulka.set_index(nu_tabulka.columns[0])

In [305]:
nu_tabulka_print

# print(nu_tabulka_print.to_latex())

,5 % H2O,8 % H2O,10 % H2O
T [°C],,,
0,12.2,12.20,12.2
100,21.3,21.42,21.5
200,31.8,32.40,32.8
300,45.0,45.48,45.8
400,59.2,59.92,60.4
500,74.6,75.62,76.3
600,91.4,92.72,93.6


#### Tabulka Pr pro daný obsah vody

In [306]:
Pr_0 = 0.69+(8-5)*((0.72-0.69)/(10-5))
Pr_100 = 0.67+(8-5)*((0.69-0.67)/(10-5))
Pr_200 = 0.65+(8-5)*((0.67-0.65)/(10-5))
Pr_300 = 0.63+(8-5)*((0.65-0.63)/(10-5))
Pr_400 = 0.62+(8-5)*((0.64-0.62)/(10-5))
Pr_500 = 0.61+(8-5)*((0.63-0.61)/(10-5))
Pr_600 = 0.60+(8-5)*((0.62-0.60)/(10-5))

Pr_tabulka = pd.DataFrame(
{
        "T [°C]": (0,100,200,300,400,500,600),
        "5 % H2O": (0.69,0.67,0.65,0.63,0.62,0.61,0.60),
        "8 % H2O": (Pr_0,Pr_100,Pr_200,Pr_300,Pr_400,Pr_500,Pr_600),
        "10 % H2O": (0.72,0.69,0.67,0.65,0.64,0.63,0.62),
    }
)
Pr_tabulka_print = Pr_tabulka.set_index(Pr_tabulka.columns[0])

In [307]:
Pr_tabulka_print

# print(Pr_tabulka_print.to_latex())

,5 % H2O,8 % H2O,10 % H2O
T [°C],,,
0,0.69,0.708,0.72
100,0.67,0.682,0.69
200,0.65,0.662,0.67
300,0.63,0.642,0.65
400,0.62,0.632,0.64
500,0.61,0.622,0.63
600,0.60,0.612,0.62


In [308]:
lambda_AB = lambda_interpolate(t_AB)
nu_AB = nu_interpolate(t_AB)
Pr_AB = Pr_interpolate(t_AB)

In [309]:
lambda_AB = 0.057
nu_AB = 8.937*10**-5
Pr_AB = 0.707

In [310]:
%%render
lambda_AB
nu_AB
Pr_AB

<IPython.core.display.Latex object>

Výpočet korekčního součinitele $c_s$

In [390]:
%%render
sigma_1 = s_1/D

<IPython.core.display.Latex object>

In [360]:
%%render
sigma_2 = s_2/D

<IPython.core.display.Latex object>

In [392]:
%%render
sigma_2_carka = sqrt(((sigma_1)**2/4 + (sigma_2)**2))

<IPython.core.display.Latex object>

In [394]:
%%render
phi_sigma = (sigma_1-1)/(sigma_2_carka-1)

<IPython.core.display.Latex object>

In [398]:
# cs = Oprava na uspořádání svazku (pro phi_sigma € <0,1 - 1,7>)
# Oprava na počet podélných řad, pro víc jak 10 trubek je C_z = 1

In [399]:
%%render
c_s = 0.34*phi_sigma**0.1
c_z = 1

<IPython.core.display.Latex object>

In [401]:
%%render
alpha_k = c_s*c_z* (lambda_AB/D) * (w_sp*D/nu_AB)**0.6 * Pr_AB**0.33    #W/m2K

<IPython.core.display.Latex object>

Součinitel přestupu tepla sáláním $\alpha_{sal}$

In [403]:
%%render
# strana 621 ČERNÝ
# epsilon_st = Součinitel emisivity povrchu výhřevné plochy
# epsilon_s = Součinitel emisivity spalin
# T_AB_s = teplota stěny je o 25 °C vyšší než teplota spalin
# epsilon_s = 0.074 z konzultací
epsilon_st = 0.82
epsilon_s = 0.074
T_AB = t_AB + 273.15    #K
T_st = t_21 + 273.15 + 25     #K

<IPython.core.display.Latex object>

In [405]:
%%render
alpha_sal = 5.68*10**-8 * (epsilon_st+1)/2 * epsilon_s * T_AB**3 * ( (1-(T_st/T_AB)**3.6) / (1-T_st/T_AB))    #W/m2K

<IPython.core.display.Latex object>

In [407]:
%%render
alpha_1r = alpha_k + alpha_sal  #(W/m2K)

<IPython.core.display.Latex object>

### Výpočet součinitele přestupu tepla ze strany páry

Určení termofyzikálních vlastností páry

In [408]:
%%render params

lambda_21 = steam.tc_pt(p_21*10, t_21)  #W/mK
mu_21 = steam.my_pt(p_21*10, t_21)      #Pas
Pr_21 = steam.pr_pt(p_21*10, t_21)      #-
nu_21 = mu_21*v_21                      #m2/s

<IPython.core.display.Latex object>

In [410]:
%%render
alpha_2r = 0.023*(lambda_21/d)*((w_p*d)/nu_21)**0.8*Pr_21**0.4*c_t*c_l*c_m  #W/m2K

<IPython.core.display.Latex object>

Korekce

In [411]:
alpha_2r = 884.43   #W/m2K

### Součinitel prostupu tepla

In [412]:
%%render
# součinitel tepelné efektivnosti (Pro plyny je 0.85 na všech plochách. DLOUHÝ, str. 125)
psi = 0.85

<IPython.core.display.Latex object>

In [414]:
%%render
k = (psi*alpha_1r)/(1+alpha_1r/alpha_2r)  #W/m2K

<IPython.core.display.Latex object>

In [415]:
k = 73.8

### Počet řad přehříváku 2

Rozdíly teplot pro výpočet středního logaritmického spádu:

In [417]:
%%render
Delta_t_1 = t_A-t_1
Delta_t_2 = t_B-t_2

<IPython.core.display.Latex object>

Střední logaritmický teplotní spád

In [418]:
%%render
Delta_t_LN = (Delta_t_1-Delta_t_2)/log(Delta_t_1/Delta_t_2) #K

<IPython.core.display.Latex object>

Vnější teplosměnná plocha

In [421]:
%%render
S_SP = (1000*Q_P2)/(k*Delta_t_LN)   #m2

<IPython.core.display.Latex object>

Vnější povrch jedné 1 m dlouhé trubky

In [423]:
%%render
S_1m = pi*D #m2

<IPython.core.display.Latex object>

Počet řad

In [425]:
%%render
n_rad = S_SP/(H*S_1m*n_tr)

<IPython.core.display.Latex object>

In [426]:
n_rad = round(n_rad,0)

### Skutečné hodnoty

Plocha

In [428]:
%%render
S_SP_skut = n_rad*n_tr*H*S_1m   #m2

<IPython.core.display.Latex object>

Teplo

In [430]:
%%render
Q_P2_skut = (k*S_SP_skut*Delta_t_LN)/1000   #kW

<IPython.core.display.Latex object>

Kontrola (rozdíl tepel nesmí překročit 5 %)

In [437]:
%%render
Delta_Qcon = 100*(abs((Q_P2_skut-Q_P2)/Q_P2_skut))    #%

<IPython.core.display.Latex object>

#### Skutečné parametry spalin v bodě B

In [433]:
%%render
I_B_skut = I_A_sp - Q_P2_skut/(V_sp*(1-z_s))    #kJ/m3

<IPython.core.display.Latex object>

In [435]:
%%render
T_B_skut = 500 + (600-500)*((I_B_skut-entalpie_sp(500,'m3n')['sp'])/(entalpie_sp(600,'m3n')['sp']-entalpie_sp(500,'m3n')['sp']))    #°C

<IPython.core.display.Latex object>

Kontrola - teploty se nesmí lišit o více jak 3°C

In [439]:
%%render
Delta_Tcon = abs(T_B_skut-t_B)  #°C

<IPython.core.display.Latex object>

In [440]:
MEZERA = ''
material = '10CrMo9-10'
material_z = '-'
data_P2 = {
    'Hodnota':
    [1000*D,1000*t_tr,1000*d,1000*h_z,1000*t_z,1000*s_z, n_z,1000*s_1,1000*s_2,S_1m,S_SP, MEZERA,alpha_k,psi,alpha_2r,k,Delta_t_LN,MEZERA,Delta_Qcon,Delta_Tcon,MEZERA,n_tr,n_rad,material,material_z],

    'Jednotka': 
    ['mm','mm','mm','mm','mm','mm','-','mm','mm','m2','m2',MEZERA,'kW/m2K','-','kW/m2K','kW/m2K','K',MEZERA,'%','K',MEZERA,'-','-','-','-']

    }
zkratky = ['D','t_tr','d','h_z','t_z','s_z','n_z','s_1','s_2','S_1m','S_SP',MEZERA,'alpha_k','psi','alpha_2r','k','Delta_t_LN',MEZERA,'Delta_Qcon','Delta_Tcon',MEZERA,'n_tr','n_rad','Materiál','Materiál žebra']
output_table = pd.DataFrame(data_P2, index = zkratky)

In [442]:
output_table

# print(output_table.to_latex())

,Hodnota,Jednotka
D,44.5,mm
t_tr,2.9,mm
d,38.7,mm
h_z,0,mm
t_z,0,mm
s_z,0,mm
n_z,0,-
s_1,88,mm
s_2,117,mm
S_1m,0.14,m2
